In [ ]:
# # INITIALIZATION
# !pip install opendatasets --quiet
# import opendatasets as od
# od.download("https://www.kaggle.com/datasets/marquis03/bean-leaf-lesions-classification")

In [ ]:
# IMPORTS

import torch
from torch import nn
from torch.optim import Adam
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# READING DATA PATHS
train_df = pd.read_csv('bean-leaf-lesions-classification/train.csv')
val_df = pd.read_csv('bean-leaf-lesions-classification/val.csv')

data_df = pd.concat([train_df, val_df], ignore_index=True)
data_df['image:FILE'] = "bean-leaf-lesions-classification/" + data_df['image:FILE']

print("Data shape is: ", data_df.shape)
print()
data_df.head()

In [ ]:
# DATA INSPECTION
print("Classes are: ")
print(data_df["category"].unique())
print()
print("Classes distrubution are: ")
print(data_df["category"].value_counts())

In [ ]:
# DATA SPLIT
train = data_df.sample(frac=0.7, random_state=42)
test = data_df.drop(train.index)
print("Train shape is: ", train.shape)
print("Test shape is: ", test.shape)

In [ ]:
# PREPROCESSING OBJECTS
from torchvision.transforms import v2

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Improved training transformations
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    # IMPORTANT: Use ImageNet normalization stats for transfer learning
    transforms.Normalize(mean=IMAGENET_MEAN,
                         std=IMAGENET_STD)
])

# Validation transforms should be minimal
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,
                         std=IMAGENET_STD),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0)
])

# v2 STYLE
NUM_CLASSES = 3
cutmix = v2.CutMix(num_classes=NUM_CLASSES, alpha=0.5)
mixup = v2.MixUp(num_classes=NUM_CLASSES, alpha=0.5)

cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

In [ ]:
# CUSTOM DATASET CLASS
from torch.utils.data import Dataset, DataLoader, default_collate
from PIL import Image
import os
from functools import partial

class AdvancedDataset(Dataset):
    """
    An advanced dataset class with several improvements:
    1. Reads data from a CSV file for more flexible annotation.
    2. Can handle different "modes" (train/val) with different transforms.
    3. Converts images to "RGB" to handle grayscale or RGBA images robustly.
    """
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
                               Expected columns: 'image_name', 'label'.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                                            on a sample.
        """
        self.annotations_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations_df)
    
    def __getitem__(self, index):
        try:
            img_name = os.path.join(self.root_dir, self.annotations_df.iloc[index, 0])
            image = Image.open(img_name).convert("RGB")
            
            # ============================
            # =====> FIX WAS HERE <=====
            # ============================
            # Return a simple Python integer, not a tensor.
            label = int(self.annotations_df.iloc[index, 1])
            # ============================

            # Apply per-image transformations
            if self.transform:
                image = self.transform(image)
                
            return image, label
        except Exception as e:
            print(f"Error loading data at index {index}: {e}")
            # Return None to be filtered by the collate_fn if needed,
            # or handle more gracefully. For now, we'll let it raise.
            raise e
    

def collate_fn_with_v2_aug(batch, v2_transform=None):
    """
    A custom collate_fn that takes a batch of data from AdvancedDataset
    and applies batch-level v2 augmentations before stacking them into a tensor.
    
    Args:
        batch (list of tuples): A list where each element is a tuple 
                                (image_tensor, label_tensor).
        v2_transform (callable, optional): A v2 transform like CutMix or MixUp
                                           to be applied to the entire batch.
    """
    # IMAGE STACKING
    images, labels = zip(*batch)
    images = torch.stack(images, 0)

    # Convert the tuple of integer labels into a single 1D tensor
    labels = torch.tensor(labels)

    if v2_transform:
        images, labels = v2_transform(images, labels)

    return images, labels


# CREATE DATASET OBJECTS
train_dataset = AdvancedDataset(csv_file='bean-leaf-lesions-classification/train.csv',
                                root_dir='bean-leaf-lesions-classification',
                                transform=train_transforms)

val_dataset = AdvancedDataset(csv_file='bean-leaf-lesions-classification/val.csv',
                              root_dir='bean-leaf-lesions-classification',
                              transform=val_transforms)

In [ ]:
# HYPERPARAMETERS AND DATALOADING
config = {
    "lr": 0.001,
    "batch_size": 64,
    "epochs": 15,
    "num_classes": 3,
    "weight_decay": 0.01
}

print("\nTraining Configuration:")
for key, value in config.items():
    print(f"  - {key}: {value}")

# --- 3. DATALOADER INSTANTIATION ---
# Now, we use these robust components to create our DataLoaders.

# Assuming 'train_dataset', 'val_dataset', and 'cutmix_or_mixup' are defined
# from the previous steps. And 'config' dictionary exists.

# Use functools.partial for a cleaner way to pass args to the collate_fn
train_collate_fn = partial(collate_fn_with_v2_aug, v2_transform=cutmix_or_mixup)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=2, # Set based on your system
    collate_fn=train_collate_fn
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn_with_v2_aug # No v2 transform needed for validation
)

In [ ]:
# googlenet_model = models.googlenet(weights='DEFAULT')
# for param in googlenet_model.parameters():
#   param.requires_grad = True

# googlenet_model.fc

In [ ]:
# MODEL TRAINING
from tqdm.autonotebook import tqdm
import time

def train_one_epoch(model, dataloader, criterion, optimizer, scheduler, device):
    """
    Execute a single training epoch.
    """
    model.train()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    lrs = []

    progress_bar = tqdm(dataloader, desc=f"Training", leave=False)
                    # tqdm(dataloader, desc=f"Epoch {mode} Progress"):

    for inputs, labels in progress_bar:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        # print(f"Train outputs size: {outputs.size()} ")
        loss = criterion(outputs, labels)
        # loss_aux1 = criterion(outputs.aux_logits1, labels)
        # loss_aux2 = criterion(outputs.aux_logits2, labels)
        # loss = loss_main + 0.3 * loss_aux1 + 0.3 * loss_aux2

        # main_preds = outputs.logits

        loss.backward()
        optimizer.step()

        
        scheduler.step()
        lrs.append(scheduler.get_last_lr()[0])

        preds = torch.argmax(outputs, 1)
        # print(f"Train preds size: {preds.size()} ")
        # print(f"Labels size: {labels.size()}")

        if labels.dim() > 1:
            true_labels = torch.argmax(labels, dim=1)
        else:
            true_labels = labels

        running_loss += loss.item() * inputs.size(0)
        running_corrects += (preds == true_labels).sum().item()
        # torch.sum(preds == labels.data)
        total_samples += labels.size(0)

        progress_bar.set_postfix(loss=loss.item(), acc=running_corrects/total_samples)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc, lrs


def evaluate(model, dataloader, criterion, device):
    """
    Evaluate the model on the validation set.
    """
    model.eval()

    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            preds = torch.argmax(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum().item()
            total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects / total_samples

    return epoch_loss, epoch_acc

In [ ]:
print("🚀 Starting training process...")

# Initialize a dictionary to store the training history for later plotting
history = {
    'train_loss': [], 'val_loss': [], 
    'train_acc': [], 'val_acc': [], 
    'lrs': []
}

model = models.googlenet(weights=None, aux_logits=True)
for param in model.parameters():
  param.requires_grad = True

model.to(device)


In [ ]:
# 1. Modify the main classifier
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, config["num_classes"])

# 2. Modify the first auxiliary classifier
num_ftrs_aux1 = model.aux1.fc2.in_features
model.aux1.fc2 = nn.Linear(num_ftrs_aux1, config["num_classes"])

# 3. Modify the second auxiliary classifier
num_ftrs_aux2 = model.aux2.fc2.in_features
model.aux2.fc2 = nn.Linear(num_ftrs_aux2, config["num_classes"])
# ============================
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), 
                 lr=config["lr"], 
                 weight_decay=config["weight_decay"]
                 )


In [ ]:
def scheduler_diff_train(scheduler):

    start = time.time()

    for epoch in range(config['epochs']):
        print(f"\nEpoch {epoch+1}/{config['epochs']}")

        train_loss, train_acc, lrs = train_one_epoch(
            model, train_loader, criterion, optimizer, scheduler, device
        )
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lrs'].extend(lrs)

        print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    end = time.time()
    total_time = end - start
    print(f"\n🏁 Training finished in {total_time:.2f} seconds.")

In [ ]:
# CosineAnnealingLR SCHEDULER
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config["epochs"]
    # steps_per_epoch=len(train_dataset), #does it change with the data augmentation
    # epochs=config["epochs"]
)

scheduler_diff_train(scheduler)

<h2>10. Promijeni kod za korištenje StepLR scheduler</h2>

In [ ]:
import copy

# --- NEW: StepLR Scheduler ---
# This scheduler will decrease the LR by a factor of 0.1 every 5 epochs.
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

print("\n✅ Model, AdamW Optimizer, and FINAL StepLR Scheduler initialized.")


# ==================================
# STAGE 2: TRAINING AND EVALUATION FUNCTIONS (UNCHANGED)
# ==================================
# The `train_one_epoch` and `evaluate` functions from the previous version are correct
# and do not need to be changed. We'll include them here for completeness.

def train_one_epoch_step(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        # loss_aux1 = criterion(outputs.aux_logits1, labels)
        # loss_aux2 = criterion(outputs.aux_logits2, labels)
        # loss = loss_main + 0.3 * loss_aux1 + 0.3 * loss_aux2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        if labels.dim() > 1:
            true_labels = torch.argmax(labels, dim=1)
        else:
            true_labels = labels
        correct_predictions += (preds == true_labels).sum().item()
        total_samples += labels.size(0)
        progress_bar.set_postfix(loss=loss.item(), acc=correct_predictions/total_samples)
    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

def evaluate_step(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluating", leave=False)
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)
            progress_bar.set_postfix(loss=loss.item(), acc=correct_predictions/total_samples)
    epoch_loss = running_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

# ==================================
# STAGE 3: MAIN TRAINING LOOP (WITH "SAVE BEST" LOGIC)
# ==================================
print("\n🚀 Starting training process with stable strategy and 'save best' logic...")
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'lrs': []}
start_time = time.time()

# Variables to track the best performing model
best_val_acc = 0.0
best_model_weights = None

for epoch in range(config["epochs"]):
    print(f"\n--- Epoch {epoch+1}/{config['epochs']} ---")

    train_loss, train_acc = train_one_epoch_step(model, train_loader, criterion, optimizer, device)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    
    val_loss, val_acc = evaluate_step(model, val_loader, criterion, device)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # The epoch-level StepLR scheduler is stepped after the validation step
    scheduler.step()
    history['lrs'].append(optimizer.param_groups[0]['lr'])
    
    print(f"Epoch Summary: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} | "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")
          
    # ============================
    # =====> NEW LOGIC HERE <=====
    # ============================
    # Check if the current model is the best one we've seen so far
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        # Save a deep copy of the model's weights
        best_model_weights = copy.deepcopy(model.state_dict())
        print(f"🎉 New best model saved with validation accuracy: {best_val_acc:.4f}")
    # ============================

end_time = time.time()
total_time = end_time - start_time
print(f"\n🏁 Training finished in {total_time:.2f} seconds.")
print(f"🏆 Best validation accuracy achieved: {best_val_acc:.4f}")

# After the loop, you can load the best weights back into the model for final evaluation
if best_model_weights:
    model.load_state_dict(best_model_weights)
    print("\n✅ Best model weights loaded for final analysis.")


<h4>Idemo sa StepLR i checkpointanjem dalje<h4>

In [ ]:
# =============================================================================
#
#       FINAL ANALYSIS, VISUALIZATION, AND SAVING SUITE
#
# This script consumes the results from the training process to:
# 1. Plot detailed training curves (loss, accuracy, learning rate).
# 2. Save the best performing model's weights and configuration to a file.
# 3. Provide an example of how to load the saved model for future use.
#
# =============================================================================


# ==================================
# STAGE 1: PLOT TRAINING & VALIDATION CURVES
# ==================================
def plot_training_history(history):
    """
    Plots the training and validation loss/accuracy, as well as the 
    learning rate schedule from a training history dictionary.
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plotting Loss
    axes[0].plot(history['train_loss'], label='Training Loss')
    axes[0].plot(history['val_loss'], label='Validation Loss')
    axes[0].set_title('Loss vs. Epochs')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)

    # Plotting Accuracy
    axes[1].plot(history['train_acc'], label='Training Accuracy')
    axes[1].plot(history['val_acc'], label='Validation Accuracy')
    axes[1].set_title('Accuracy vs. Epochs')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()


print("📈 Plotting training history...")
plot_training_history(history)


# ==================================
# STAGE 2: SAVE THE BEST MODEL AND PARAMETERS
# ==================================

# First, ensure the model has the best weights loaded
if best_model_weights:
    model.load_state_dict(best_model_weights)
    print("\n✅ Best model weights have been loaded into the model.")

# Define a path for saving the model
save_path = "best_model.pth"


# Create a dictionary to save everything needed for reproducibility
checkpoint = {
    'config': config,
    'model_state_dict': model.state_dict(),
    'best_val_acc': best_val_acc # From the training loop
}

# Save the checkpoint dictionary to the file
torch.save(checkpoint, save_path)
print(f"💾 Model and configuration saved successfully to '{save_path}'")



# ==================================
# STAGE 3: EXAMPLE OF LOADING THE SAVED MODEL
# ==================================
# This demonstrates how you would use your saved model in a different script.

def load_model_for_inference(path):
    """Loads a saved checkpoint for inference."""
    
    # Load the saved checkpoint
    checkpoint = torch.load(path)
    
    # Get the saved configuration
    saved_config = checkpoint['config']
    
    # Re-create the model architecture using the saved config
    loaded_model = models.get_model(saved_config["model_name"], pretrained=False) # No need to download weights
    num_ftrs = loaded_model.fc.in_features
    loaded_model.fc = nn.Linear(num_ftrs, saved_config["num_classes"])
    
    # Load the saved weights into the model
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    
    print("\n✅ Model loaded successfully from checkpoint.")
    print(f"   - Original Model: {saved_config['model_name']}")
    print(f"   - Trained for: {saved_config['epochs']} epochs")
    print(f"   - Achieved Val Acc: {checkpoint['best_val_acc']:.4f}")
    
    return loaded_model

# Demonstrate loading the model we just saved
inference_model = load_model_for_inference(save_path)
